# Mecanismos de atenção e transformadores

Uma grande desvantagem das redes recorrentes é que todas as palavras em uma sequência têm o mesmo impacto no resultado. Isso causa um desempenho subótimo em modelos padrão de codificador-decodificador LSTM para tarefas de sequência para sequência, como Reconhecimento de Entidades Nomeadas e Tradução Automática. Na realidade, palavras específicas na sequência de entrada frequentemente têm mais impacto nos resultados sequenciais do que outras.

Considere um modelo de sequência para sequência, como tradução automática. Ele é implementado por duas redes recorrentes, onde uma rede (**codificador**) comprime a sequência de entrada em um estado oculto, e outra (**decodificador**) expande esse estado oculto no resultado traduzido. O problema com essa abordagem é que o estado final da rede tem dificuldade em lembrar o início de uma frase, causando baixa qualidade do modelo em frases longas.

**Mecanismos de Atenção** fornecem um meio de ponderar o impacto contextual de cada vetor de entrada em cada previsão de saída da RNN. Isso é implementado criando atalhos entre estados intermediários da RNN de entrada e da RNN de saída. Dessa forma, ao gerar o símbolo de saída $y_t$, levaremos em conta todos os estados ocultos de entrada $h_i$, com diferentes coeficientes de peso $\alpha_{t,i}$.

![Imagem mostrando um modelo codificador/decodificador com uma camada de atenção aditiva](../../../../../translated_images/pt-BR/encoder-decoder-attention.7a726296894fb567.webp)
*O modelo codificador-decodificador com mecanismo de atenção aditiva em [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citado deste [post de blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

A matriz de atenção $\{\alpha_{i,j}\}$ representaria o grau em que certas palavras de entrada influenciam a geração de uma palavra específica na sequência de saída. Abaixo está um exemplo de tal matriz:

![Imagem mostrando um alinhamento de exemplo encontrado pelo RNNsearch-50, retirada de Bahdanau - arviz.org](../../../../../translated_images/pt-BR/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figura retirada de [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Os mecanismos de atenção são responsáveis por grande parte do estado da arte atual ou próximo ao estado da arte em Processamento de Linguagem Natural. No entanto, adicionar atenção aumenta significativamente o número de parâmetros do modelo, o que levou a problemas de escalabilidade com RNNs. Uma restrição chave na escalabilidade das RNNs é que a natureza recorrente dos modelos torna desafiador agrupar e paralelizar o treinamento. Em uma RNN, cada elemento de uma sequência precisa ser processado em ordem sequencial, o que significa que não pode ser facilmente paralelizado.

A adoção de mecanismos de atenção combinada com essa restrição levou à criação dos modelos transformadores, agora estado da arte, que conhecemos e usamos hoje, como BERT e OpenGPT3.

## Modelos transformadores

Em vez de encaminhar o contexto de cada previsão anterior para a próxima etapa de avaliação, **modelos transformadores** usam **codificações posicionais** e atenção para capturar o contexto de uma entrada específica dentro de uma janela de texto fornecida. A imagem abaixo mostra como as codificações posicionais com atenção podem capturar o contexto dentro de uma janela específica.

![GIF animado mostrando como as avaliações são realizadas em modelos transformadores.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Como cada posição de entrada é mapeada independentemente para cada posição de saída, os transformadores podem paralelizar melhor do que as RNNs, o que permite modelos de linguagem muito maiores e mais expressivos. Cada cabeça de atenção pode ser usada para aprender diferentes relações entre palavras, melhorando tarefas de Processamento de Linguagem Natural.

**BERT** (Representações de Codificador Bidirecional de Transformadores) é uma rede transformadora muito grande e multilayer com 12 camadas para *BERT-base* e 24 para *BERT-large*. O modelo é primeiro pré-treinado em um grande corpus de dados de texto (WikiPedia + livros) usando treinamento não supervisionado (prevendo palavras mascaradas em uma frase). Durante o pré-treinamento, o modelo absorve um nível significativo de compreensão da linguagem, que pode ser aproveitado com outros conjuntos de dados usando ajuste fino. Esse processo é chamado de **aprendizado por transferência**.

![Imagem de http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/pt-BR/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Existem muitas variações de arquiteturas de transformadores, incluindo BERT, DistilBERT, BigBird, OpenGPT3 e outras que podem ser ajustadas. O pacote [HuggingFace](https://github.com/huggingface/) fornece um repositório para treinar muitas dessas arquiteturas com PyTorch.

## Usando BERT para classificação de texto

Vamos ver como podemos usar o modelo BERT pré-treinado para resolver nossa tarefa tradicional: classificação de sequência. Vamos classificar nosso conjunto de dados original AG News.

Primeiro, vamos carregar a biblioteca HuggingFace e nosso conjunto de dados:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Como estaremos utilizando um modelo BERT pré-treinado, será necessário usar um tokenizador específico. Primeiro, vamos carregar um tokenizador associado ao modelo BERT pré-treinado.

A biblioteca HuggingFace contém um repositório de modelos pré-treinados, que você pode usar simplesmente especificando seus nomes como argumentos para as funções `from_pretrained`. Todos os arquivos binários necessários para o modelo serão baixados automaticamente.

No entanto, em determinados momentos, pode ser necessário carregar seus próprios modelos. Nesse caso, você pode especificar o diretório que contém todos os arquivos relevantes, incluindo os parâmetros para o tokenizador, o arquivo `config.json` com os parâmetros do modelo, os pesos binários, etc.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

O objeto `tokenizer` contém a função `encode` que pode ser usada diretamente para codificar texto:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Então, vamos criar iteradores que usaremos durante o treinamento para acessar os dados. Como o BERT usa sua própria função de codificação, precisaríamos definir uma função de preenchimento semelhante à `padify` que definimos anteriormente:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

No nosso caso, usaremos o modelo BERT pré-treinado chamado `bert-base-uncased`. Vamos carregar o modelo usando o pacote `BertForSequenceClassfication`. Isso garante que nosso modelo já tenha a arquitetura necessária para classificação, incluindo o classificador final. Você verá uma mensagem de aviso indicando que os pesos do classificador final não estão inicializados e que o modelo precisará de pré-treinamento - isso é perfeitamente normal, porque é exatamente o que estamos prestes a fazer!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Agora estamos prontos para começar o treinamento! Como o BERT já está pré-treinado, queremos começar com uma taxa de aprendizado relativamente pequena para não comprometer os pesos iniciais.

Todo o trabalho pesado é realizado pelo modelo `BertForSequenceClassification`. Quando chamamos o modelo nos dados de treinamento, ele retorna tanto a perda quanto a saída da rede para o minibatch de entrada. Usamos a perda para a otimização dos parâmetros (`loss.backward()` realiza a passagem para trás) e `out` para calcular a precisão do treinamento, comparando os rótulos obtidos `labs` (calculados usando `argmax`) com os rótulos esperados `labels`.

Para controlar o processo, acumulamos a perda e a precisão ao longo de várias iterações e as exibimos a cada ciclo de treinamento definido por `report_freq`.

Este treinamento provavelmente levará bastante tempo, então limitamos o número de iterações.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Você pode perceber (especialmente se aumentar o número de iterações e esperar tempo suficiente) que a classificação com BERT nos oferece uma precisão muito boa! Isso acontece porque o BERT já entende muito bem a estrutura da linguagem, e só precisamos ajustar o classificador final. No entanto, como o BERT é um modelo grande, todo o processo de treinamento leva bastante tempo e exige um poder computacional significativo! (GPU, e de preferência mais de uma).

> **Note:** Em nosso exemplo, estamos utilizando um dos menores modelos BERT pré-treinados. Existem modelos maiores que provavelmente produzirão resultados melhores.


## Avaliando o desempenho do modelo

Agora podemos avaliar o desempenho do nosso modelo no conjunto de dados de teste. O loop de avaliação é bem semelhante ao loop de treinamento, mas não devemos esquecer de alternar o modelo para o modo de avaliação chamando `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Conclusão

Nesta unidade, vimos como é fácil utilizar um modelo de linguagem pré-treinado da biblioteca **transformers** e adaptá-lo para nossa tarefa de classificação de texto. Da mesma forma, os modelos BERT podem ser usados para extração de entidades, resposta a perguntas e outras tarefas de PLN.

Os modelos de transformadores representam o estado da arte atual em PLN e, na maioria dos casos, devem ser a primeira solução com a qual você começa a experimentar ao implementar soluções personalizadas de PLN. No entanto, compreender os princípios básicos subjacentes às redes neurais recorrentes discutidos neste módulo é extremamente importante se você deseja construir modelos neurais avançados.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autoritativa. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
